In [1]:
import torch
from torchvision import datasets,models
import torchvision.transforms as tt
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torch.utils.data import DataLoader,random_split,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from fastprogress.fastprogress import master_bar, progress_bar

In [2]:
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
train_tfms = tt.Compose([tt.Resize(256),
                         tt.CenterCrop(224),
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)])
valid_tfms = tt.Compose([tt.Resize([224,224]),tt.ToTensor(), tt.Normalize(*stats)])

In [3]:
ds = datasets.ImageFolder('food-101-subset/images/')

In [4]:
class CustomDataset(Dataset):
    def __init__(self,ds,transformer):
        self.ds = ds
        self.transform = transformer
    
    def __getitem__(self,idx):
        image,label = self.ds[idx]
        img = self.transform(image)
        return img,label
    
    def __len__(self):
        return len(self.ds)

In [5]:
train_len=0.8*len(ds)
val_len = len(ds) - train_len
int(train_len),int(val_len)

(16160, 4040)

In [6]:
train_ds,val_ds = random_split(dataset=ds,lengths=[int(train_len),int(val_len)],generator=torch.Generator().manual_seed(42))

In [7]:
t_ds = CustomDataset(train_ds.dataset,train_tfms)
v_ds = CustomDataset(val_ds.dataset,valid_tfms)

In [8]:
batch_size = 32
train_dl = DataLoader(t_ds, batch_size, shuffle=True, pin_memory=True)
valid_dl = DataLoader(v_ds, batch_size, pin_memory=True)

In [9]:
class Flatten(nn.Module):
    def forward(self,x):
        return torch.flatten(x,1)

class FoodImageClassifer(nn.Module):
    def __init__(self):
        super().__init__()
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.body = mobilenet.features
        self.head = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(1280,101))
    
    def forward(self,x):
        x = self.body(x)
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1) 
        return self.head(x)
    
    def freeze(self):
        for name,param in self.body.named_parameters():
            param.requires_grad = False

In [10]:
def fit(epochs,model,train_dl,valid_dl,loss_fn,opt):
    mb = master_bar(range(epochs))
    mb.write(['epoch','train_loss','valid_loss','trn_acc','val_acc'],table=True)

    for i in mb:    
        trn_loss,val_loss = 0.0,0.0
        trn_acc,val_acc = 0,0
        trn_n,val_n = len(train_dl.dataset),len(valid_dl.dataset)
        model.train()
        for xb,yb in progress_bar(train_dl,parent=mb):
            xb,yb = xb.to(device), yb.to(device)
            out = model(xb)
            opt.zero_grad()
            loss = loss_fn(out,yb)
            _,pred = torch.max(out.data, 1)
            trn_acc += (pred == yb).sum().item()
            trn_loss += loss.item()
            loss.backward()
            opt.step()
        trn_loss /= mb.child.total
        trn_acc /= trn_n

        model.eval()
        with torch.no_grad():
            for xb,yb in progress_bar(valid_dl,parent=mb):
                xb,yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = loss_fn(out,yb)
                val_loss += loss.item()
                _,pred = torch.max(out.data, 1)
                val_acc += (pred == yb).sum().item()
        val_loss /= mb.child.total
        val_acc /= val_n

        mb.write([i,f'{trn_loss:.6f}',f'{val_loss:.6f}',f'{trn_acc:.6f}',f'{val_acc:.6f}'],table=True)

In [17]:
model = FoodImageClassifer()
criterion = nn.CrossEntropyLoss()
optimizer_ft =  optim.Adam(model.parameters(), lr=1e-4)
#model.freeze()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
fit(1,model=model,train_dl=train_dl,valid_dl=valid_dl,loss_fn=criterion,opt=optimizer_ft)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,2.816711,1.739123,0.374653,0.598564


In [18]:
fit(5,model=model,train_dl=train_dl,valid_dl=valid_dl,loss_fn=criterion,opt=optimizer_ft)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,1.551284,1.188755,0.626931,0.705891
1,1.090450,0.957794,0.731832,0.758317
2,0.773461,0.748548,0.806980,0.805099
3,0.533341,0.640974,0.871089,0.824455
4,0.357086,0.563184,0.919455,0.842871


In [20]:
torch.save(model.state_dict(),'food_classifier.pth')

# Test Loop

In [26]:
model = FoodImageClassifer()

In [28]:
model.load_state_dict(torch.load('food_classifier_progressive.pth'))

<All keys matched successfully>

In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [30]:
test_ds = datasets.ImageFolder('food-101/images/',valid_tfms)

In [31]:
batch_size = 32
test_dl = DataLoader(test_ds, batch_size, pin_memory=True)

In [32]:
def test_fit(epochs,model,test_dl,loss_fn):
    mb = master_bar(range(epochs))
    mb.write(['epoch','test_loss','test_acc'],table=True)

    for i in mb:    
        test_loss = 0.0
        test_acc = 0
        test_n = len(test_dl.dataset)
        model.eval()
        with torch.no_grad():
            for xb,yb in progress_bar(test_dl,parent=mb):
                xb,yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = loss_fn(out,yb)
                test_loss += loss.item()
                _,pred = torch.max(out.data, 1)
                test_acc += (pred == yb).sum().item()
        test_loss /= mb.child.total
        test_acc /= test_n

        mb.write([i,f'{test_loss:.6f}',f'{test_acc:.6f}'],table=True)

In [ ]:
test_fit(1,model,test_dl,criterion)

epoch,test_loss,test_acc


# Increasing Accuracy

In [12]:
model = FoodImageClassifer()
model.load_state_dict(torch.load('food_classifier.pth'))

In [23]:
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
train_tfms = tt.Compose([tt.Resize(224),
                         tt.CenterCrop(224),
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)])
valid_tfms = tt.Compose([tt.Resize([224,224]),tt.ToTensor(), tt.Normalize(*stats)])
t_ds = CustomDataset(train_ds.dataset,train_tfms)
v_ds = CustomDataset(val_ds.dataset,valid_tfms)
batch_size = 32
train_dl = DataLoader(t_ds, batch_size, shuffle=True, pin_memory=True)
valid_dl = DataLoader(v_ds, batch_size, pin_memory=True)

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer_ft =  optim.Adam(model.parameters(), lr=1e-4)
#model.freeze()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
fit(5,model=model,train_dl=train_dl,valid_dl=valid_dl,loss_fn=criterion,opt=optimizer_ft)

epoch,train_loss,valid_loss,trn_acc,val_acc
0,0.630665,0.304795,0.816386,0.917277
1,0.333461,0.208778,0.911584,0.948119
2,0.206665,0.159460,0.951535,0.959505
3,0.136959,0.131007,0.971139,0.966337
4,0.101081,0.132577,0.978911,0.964109


In [25]:
torch.save(model.state_dict(),'food_classifier_progressive.pth')